<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/Organize_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rename the dataset
This program will organize the dataset from the Raw_Dataset folder in Colab Notebooks/Datasets/OurDataset/Raw_Dataset and put them in the Experiment folder in Colab Notebooks/Datgasets/OurDataset/Raw_Dataset_Renamed.

The program will copy all the files extracting only the located hand images, resizing the images into 224x224, and renaming the images to their class name.

In [1]:
!pip install mediapipe

     |████████████████████████████████| 30.1 MB 12 kB/s 


In [2]:
import os
import cv2
import mediapipe as mp

In [4]:
dataset_raw = '/content/drive/MyDrive/Colab Notebooks/Datasets/OurDataset/Raw_Dataset'
dataset_raw_renamed = '/content/drive/MyDrive/Colab Notebooks/Datasets/OurDataset/Raw_Dataset_Renamed'


In [5]:
letters = ['A', 'B', 'C', 'D', 'E',
           'F', 'G', 'H', 'I', 'J',
           'K', 'L', 'M', 'N', 'O',
           'P', 'Q', 'R', 'S', 'T',
           'U', 'V', 'W', 'X', 'Y',
           'Z']
numbers = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [6]:
def make_dir(path, class_obj):
  for obj in class_obj:
    path_class = os.path.join(path, obj)
    os.makedirs(path_class)

In [7]:
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
if os.path.isdir(os.path.join(dataset_raw_renamed_letters, 'A')) is False:
  make_dir(dataset_raw_renamed_letters, letters)
else:
  print('Path already exist')

In [16]:
class HandDetector():
    def __init__(self, mode=False, maxHands=2, detectionConf=0.5, trackingConf=0.5):
        # Arguments
        self.mode = mode
        self.maxHands = maxHands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        # Initialize mediapipe variables
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,
                                        self.detectionConf, self.trackingConf)
        self.mpDraw = mp.solutions.drawing_utils


    def find_hands(self, src_img, draw=False):
        # Convert frame to rgb for mediapipe
        rgb = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)

        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        rgb.flags.writeable = False
        self.results = self.hands.process(rgb)

        if self.results.multi_hand_landmarks:
            x_pts, y_pts = [], []
            for handLMs in self.results.multi_hand_landmarks:
                for id, lm in enumerate(handLMs.landmark):
                    h, w, c = src_img.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    x_pts.append(cx)
                    y_pts.append(cy)
                if draw:
                    self.mpDraw.draw_landmarks(src_img, handLMs, self.mpHands.HAND_CONNECTIONS)

            # Find the max and min points
            y_max, y_min, x_max, x_min = max(y_pts), min(y_pts), max(x_pts), min(x_pts)
            extra_space = src_img.shape[0] * 0.14
            return True, (x_min - extra_space, y_max + extra_space), (x_max + extra_space, y_min - extra_space)

        return False, (), ()

In [10]:
detector = HandDetector()

In [11]:
def resize_image(src_img, img_size=(224, 224)):
  return cv2.resize(src_img, img_size, interpolation=cv2.INTER_CUBIC)

Take the images from Raw_Dataset.

Extract the hands from each images.

Make a copy and save them to Raw_Dataset_Renamed.

Name them from 0 to N number of images.


In [20]:
# Letters
dataset_raw_letters = os.path.join(dataset_raw, 'Letters')
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
for letter in letters:
  path_class = os.path.join(dataset_raw_letters, letter)
  path_class_renamed = os.path.join(dataset_raw_renamed_letters, letter)
  i = 0
  for item in os.listdir(path_class):
    img = cv2.imread(os.path.join(path_class, item))
    detected, pts_upper_left, pts_lower_right = detector.find_hands(img)
    if detected:
      file_name = letter + "_" + str(i) + ".jpg"
      roi = img[abs(int(pts_lower_right[1])):abs(int(pts_upper_left[1])), abs(int(pts_upper_left[0])):abs(int(pts_lower_right[0]))]
      roi = resize_image(roi)
      cv2.imwrite(os.path.join(path_class_renamed, file_name), roi)
      i += 1


Count the total number of images in the experiment and raw dataset folders

In [21]:
# Raw Dataset Folder

# letters
dataset_raw_letters = os.path.join(dataset_raw, 'Letters')
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
for letter in letters:
  path_class = os.path.join(dataset_raw_letters, letter)
  print(letter, len(os.listdir(path_class)))

A 96
B 97
C 100
D 100
E 102
F 98
G 100
H 100
I 100
J 0
K 100
L 97
M 95
N 91
O 92
P 92
Q 91
R 91
S 91
T 91
U 91
V 100
W 100
X 100
Y 100
Z 0


In [22]:
# Raw Dataset Renamed Folder

# letters
dataset_raw_letters = os.path.join(dataset_raw, 'Letters')
dataset_raw_renamed_letters = os.path.join(dataset_raw_renamed, 'Letters')
for letter in letters:
  path_class_renamed = os.path.join(dataset_raw_renamed_letters, letter)
  print(letter, len(os.listdir(path_class_renamed)))

A 94
B 93
C 90
D 94
E 98
F 94
G 93
H 90
I 95
J 0
K 97
L 92
M 90
N 87
O 89
P 87
Q 85
R 83
S 87
T 86
U 82
V 95
W 96
X 93
Y 99
Z 0
